In [1]:
import os

In [2]:
%pwd

'c:\\Users\\HighFresh\\Desktop\\My_Project\\End-to-end-ML-Project-Implementation\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\HighFresh\\Desktop\\My_Project\\End-to-end-ML-Project-Implementation'

In [5]:
from dataclasses import dataclass
from pathlib import Path

#This class is use to create entity just like configBox
@dataclass(frozen=True) #This is use to enable the entity 
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        # self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

config\config.yaml
[2023-10-20 21:29:31,187: INFO: common: yaml file: config\config.yaml loaded successfully]
params.yaml
[2023-10-20 21:29:31,211: INFO: common: yaml file: params.yaml loaded successfully]
[2023-10-20 21:29:31,217: INFO: common: created directory at: artifacts]
[2023-10-20 21:29:31,221: INFO: common: created directory at: artifacts/data_ingestion]
[2023-10-20 21:29:37,810: INFO: 951979835: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 26255
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "fa47ae6f42117d6f7052af668f9adf9c8e231c3e7f128cef2d9cf4ae1e0e2760"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: B32E:BE5D:B3C01:BA6DF:6532E332
Accept-Ranges: bytes
Date: Fri, 20 Oct 2023 20:29:39 GMT
Via: 1.1 varnish
X-Served-By: cache